In [1]:
# import modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import streamlit  as st
from datetime import datetime

In [2]:
# Load in datasets
df_agg = pd.read_csv("Aggregated_Metrics_By_Video.csv").iloc[1:, :]
df_comments = pd.read_csv("All_Comments_Final.csv")
df_agg_sub = pd.read_csv("Aggregated_Metrics_By_Country_And_Subscriber_Status.csv")
df_time = pd.read_csv("Video_Performance_Over_Time.csv")

In [3]:
df_agg.head(2)

,Video,Video title,Video pub­lish time,Com­ments ad­ded,Shares,Dis­likes,Likes,Sub­scribers lost,Sub­scribers gained,RPM (USD),CPM (USD),Av­er­age per­cent­age viewed (%),Av­er­age view dur­a­tion,Views,Watch time (hours),Sub­scribers,Your es­tim­ated rev­en­ue (USD),Im­pres­sions,Im­pres­sions click-through rate (%)
1,4OZip0cgOho,How I Would Learn Data Science (If I Had to St...,"May 8, 2020",907,9583,942,46903,451,46904,6.353,12.835,36.65,0:03:09,1253559,65850.7042,46453,7959.533,26498799,3.14
2,78LjdAAw0wA,100K Channel Update + AMA Stream!,"Nov 12, 2020",412,4,4,130,15,12,2.668,6.259,6.26,0:05:14,2291,200.2966,-3,6.113,188318,0.72


In [4]:
# df_agg_sub.head(3)

In [5]:
df_agg.columns

Index(['Video', 'Video title', 'Video pub­lish time', 'Com­ments ad­ded',
       'Shares', 'Dis­likes', 'Likes', 'Sub­scribers lost',
       'Sub­scribers gained', 'RPM (USD)', 'CPM (USD)',
       'Av­er­age per­cent­age viewed (%)', 'Av­er­age view dur­a­tion',
       'Views', 'Watch time (hours)', 'Sub­scribers',
       'Your es­tim­ated rev­en­ue (USD)', 'Im­pres­sions',
       'Im­pres­sions click-through rate (%)'],
      dtype='object')

In [6]:
df_agg['Video pub­lish time'] = pd.to_datetime(df_agg['Video pub­lish time'])
df_agg['Av­er­age view dur­a­tion'] = df_agg['Av­er­age view dur­a­tion'].apply(lambda x: datetime.strptime(x, '%H:%M:%S'))
df_agg['Av­g_dur­a­tion_sec'] = df_agg['Av­er­age view dur­a­tion'].apply(lambda x: x.second + x.minute*60 + x.hour*3600)
df_agg['Engagement_ratio'] = (df_agg['Com­ments ad­ded'] + df_agg['Shares'] + df_agg['Likes'] + df_agg['Dis­likes']) / df_agg['Views']
df_agg['Views/Sub Gained'] = df_agg['Views'] / df_agg['Sub­scribers gained']
df_agg.sort_values('Video pub­lish time', ascending=False, inplace= True)

In [7]:
df_time['Date'] = pd.to_datetime(df_time['Date'])

In [8]:
# create a fxn to engineer data
# @st.cache
def load_data():
    # Load in datasets
    df_agg = pd.read_csv("Aggregated_Metrics_By_Video.csv").iloc[1:, :]
    df_comments = pd.read_csv("All_Comments_Final.csv")
    df_agg_sub = pd.read_csv("Aggregated_Metrics_By_Country_And_Subscriber_Status.csv")
    df_time = pd.read_csv("Video_Performance_Over_Time.csv")

    # transform data
    df_agg['Video pub­lish time'] = pd.to_datetime(df_agg['Video pub­lish time'])
    df_agg['Av­er­age view dur­a­tion'] = df_agg['Av­er­age view dur­a­tion'].apply(lambda x: datetime.strptime(x, '%H:%M:%S'))
    df_agg['Av­g_dur­a­tion_sec'] = df_agg['Av­er­age view dur­a­tion'].apply(lambda x: x.second + x.minute*60 + x.hour*3600)
    df_agg['Engagement_ratio'] = (df_agg['Com­ments ad­ded'] + df_agg['Shares'] + df_agg['Likes'] + df_agg['Dis­likes']) / df_agg['Views']
    df_agg['Views/Sub Gained'] = df_agg['Views'] / df_agg['Sub­scribers gained']
    df_agg.sort_values('Video pub­lish time', ascending=False, inplace= True)
    df_time['Date'] = pd.to_datetime(df_time['Date'])

    return df_agg, df_agg_sub, df_comments, df_time

In [9]:
df_agg_diff = df_agg.copy()

In [10]:
# data enigneering
metric_date_12mo = df_agg_diff['Video pub­lish time'].max() - pd.DateOffset(months=12)
median_agg = df_agg_diff[df_agg_diff['Video pub­lish time'] >= metric_date_12mo].median()

numeric_cols = np.array((df_agg_diff.dtypes == 'float64') | (df_agg_diff.dtypes == 'int64'))
df_agg_diff.iloc[:,numeric_cols] = (df_agg_diff.iloc[:,numeric_cols] - median_agg).div(median_agg)

C:\Users\Abbey Christian\AppData\Local\Temp\ipykernel_5328\3125710640.py:3: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  median_agg = df_agg_diff[df_agg_diff['Video pub­lish time'] >= metric_date_12mo].median()
C:\Users\Abbey Christian\AppData\Local\Temp\ipykernel_5328\3125710640.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  median_agg = df_agg_diff[df_agg_diff['Video pub­lish time'] >= metric_date_12mo].median()


In [11]:
add_sidebar = st.sidebar.selectbox('Aggregate or Individual Video', ('Aggregate Metrics', 'Individual Video Analysis'))

2023-03-26 19:43:08.753 
  command:

    streamlit run D:\anaconda\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [12]:
df_agg.columns

Index(['Video', 'Video title', 'Video pub­lish time', 'Com­ments ad­ded',
       'Shares', 'Dis­likes', 'Likes', 'Sub­scribers lost',
       'Sub­scribers gained', 'RPM (USD)', 'CPM (USD)',
       'Av­er­age per­cent­age viewed (%)', 'Av­er­age view dur­a­tion',
       'Views', 'Watch time (hours)', 'Sub­scribers',
       'Your es­tim­ated rev­en­ue (USD)', 'Im­pres­sions',
       'Im­pres­sions click-through rate (%)', 'Av­g_dur­a­tion_sec',
       'Engagement_ratio', 'Views/Sub Gained'],
      dtype='object')

In [52]:
if add_sidebar == 'Aggregate Metrics':
    df_agg_metrics =df_agg[['Video pub­lish time', 'Com­ments ad­ded',
       'Shares', 'Dis­likes', 'Likes', 'Sub­scribers lost',
       'Sub­scribers gained', 'RPM (USD)', 'CPM (USD)',
       'Av­er­age per­cent­age viewed (%)', 'Av­er­age view dur­a­tion',
       'Views', 'Watch time (hours)', 'Sub­scribers',
       'Your es­tim­ated rev­en­ue (USD)', 'Im­pres­sions',
       'Im­pres­sions click-through rate (%)', 'Av­g_dur­a­tion_sec',
       'Engagement_ratio', 'Views/Sub Gained']]
    metric_date_6mo = df_agg_metrics['Video pub­lish time'].max() - pd.DateOffset(months=6)
    metric_date_12mo = df_agg_metrics['Video pub­lish time'].max() - pd.DateOffset(months=12)
    metric_median_6mo = df_agg_metrics[df_agg_metrics['Video pub­lish time'] >= metric_date_6mo].median()
    metric_median_12mo = df_agg_metrics[df_agg_metrics['Video pub­lish time'] >= metric_date_12mo].median()
    
    col1, col2, col3, col4, col5 = st.columns(5)
    columns = [col1, col2, col3, col4, col5]
    count = 0
    for i in metric_median_6mo:
        with columns[count]:
            delta = (metric_median_6mo[i] - metric_median_12mo[i]) / metric_median_12mo[i]
            st.metric(label=i, value=round(metric_median_6mo[i], 1), delta="{:,.2%}".format(delta))
            count += 1
            if count > 5:
                count = 0


if add_sidebar == 'Individual Video Analysis':
    st.write('Individual Analysis')

C:\Users\Abbey Christian\AppData\Local\Temp\ipykernel_5992\3942847265.py:12: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  metric_median_6mo = df_agg_metrics[df_agg_metrics['Video pub­lish time'] >= metric_date_6mo].median()
C:\Users\Abbey Christian\AppData\Local\Temp\ipykernel_5992\3942847265.py:13: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  metric_median_12mo = df_agg_metrics[df_agg_metrics['Video pub­lish time'] >= metric_date_12mo].median()


KeyError: 36.0

In [51]:
col1, col2, col3, col4, col5 = st.columns(5)
columns = [col1, col2, col3, col4, col5]
count = 0
for i in metric_median_6mo:
    with columns[count]:
        delta = (metric_median_6mo[i] - metric_median_12mo[i]) / metric_median_12mo[i]
        st.metric(label=i, value=round(metric_median_6mo[i], 1), delta="{:,.2%}".format(delta))
        count += 1
        if count > 5:
            count = 0

KeyError: 36.0

In [44]:
for i in metric_median_6mo.index:
    delta = (metric_median_6mo[i] - metric_median_12mo[i]) / metric_median_12mo[i]
    print(delta)

-0.1724137931034483
-0.03529411764705882
-0.1
-0.20261437908496732
0.0
-0.37168141592920356
0.00572082379862688
0.0
0.001214329083181646
-0.18268841849804504
-0.007147344626553326
-0.42857142857142855
0.0
-0.11160684063764285
-0.03909465020576122
0.05421686746987952
0.11326597943058413
0.04797355368027618


In [20]:
df_agg_diff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 223 entries, 111 to 223
Data columns (total 22 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Video                                 223 non-null    object        
 1   Video title                           223 non-null    object        
 2   Video pub­lish time                   223 non-null    datetime64[ns]
 3   Com­ments ad­ded                      223 non-null    float64       
 4   Shares                                223 non-null    float64       
 5   Dis­likes                             223 non-null    float64       
 6   Likes                                 223 non-null    float64       
 7   Sub­scribers lost                     223 non-null    float64       
 8   Sub­scribers gained                   223 non-null    float64       
 9   RPM (USD)                             223 non-null    float64       
 10  

In [13]:
df_agg.columns

Index(['Video', 'Video title', 'Video pub­lish time', 'Com­ments ad­ded',
       'Shares', 'Dis­likes', 'Likes', 'Sub­scribers lost',
       'Sub­scribers gained', 'RPM (USD)', 'CPM (USD)',
       'Av­er­age per­cent­age viewed (%)', 'Av­er­age view dur­a­tion',
       'Views', 'Watch time (hours)', 'Sub­scribers',
       'Your es­tim­ated rev­en­ue (USD)', 'Im­pres­sions',
       'Im­pres­sions click-through rate (%)', 'Av­g_dur­a­tion_sec',
       'Engagement_ratio', 'Views/Sub Gained'],
      dtype='object')